*Create CNN Model and Optimize it Using Keras Tuner*

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=e6c71c1f185822c60e774c380d7e8f9bcd71af424c63164f6a3b1f8008d2d59b
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=d7d5b12d76efd2089651ce2fdc9e49c9ec53e24d445115fc43c793546dbeced0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.4.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
## scaling down the images to 0 to 1. Where all pixels will be scaled from 0 to 1
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [ ]:
#create a function called build model
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv1_filter', min_value=64, max_value=128, step=16),
                                kernel_size=hp.Choice('conv1_kernel', values=[3,5]),
                                activation='relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv2_filter', min_value=32, max_value=128, step=16),
                                kernel_size=hp.Choice('conv2_kernel', values=[3,5]),
                                activation='relu',
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense1_units', min_value=64, max_value=128, step=16),
                                activation='relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')
                          ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
# Random Search will find out which parameter will be best.
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist_Fashion")

In [ ]:
#run a search function to find best pararme
tuner_search.search(train_images,train_labels,epochs=3, validation_split=0.1)

Trial 1 Complete [00h 31m 42s]
val_accuracy: 0.9006666541099548

Best val_accuracy So Far: 0.9006666541099548
Total elapsed time: 00h 31m 42s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv1_filter      |80                |80                
conv1_kernel      |3                 |5                 
conv2_filter      |64                |96                
conv2_kernel      |5                 |5                 
dense1_units      |112               |96                
learning_rate     |0.001             |0.001             

Epoch 1/3
1117/1688 [==================>...........] - ETA: 3:09 - loss: 0.5787 - accuracy: 0.7920

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
#Get the best model selscted
model.summary()

In [ ]:
#Retrain all the images with the best model that we got to know
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)